In [2]:
import pandas as pd
import sqlite3

from faker import Faker
from faker.providers import BaseProvider

In [399]:
cnx = sqlite3.connect("base_datos_proveedores.db")

In [400]:
cursor_prov = cnx.cursor()

In [5]:
query = '''
CREATE TABLE proveedores (
id_proveedor INT PRIMARY KEY,
nombre TEXT NOT NULL,
direccion TEXT NOT NULL,
ciudad TEXT NOT NULL,
provincia TEXT NOT NULL
) 
'''
cursor_prov.execute(query)

OperationalError: table proveedores already exists

In [ ]:
query = ''' 
SELECT *
FROM sqlite_master
WHERE type = "table"
'''
pd.read_sql(query, cnx)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\nid_proveedor INT P...


In [ ]:
query = ''' 
SELECT *
FROM proveedores
'''
pd.read_sql(query, cnx)

,id_proveedor,nombre,direccion,ciudad,provincia


In [ ]:
query = '''
CREATE TABLE piezas (
id_pieza INT PRIMARY KEY,
nombre TEXT NOT NULL,
color TEXT NOT NULL,
precio FLOAT NOT NULL,
categoria TEXT NOT NULL
) 
'''
cursor_prov.execute(query)

In [ ]:
query = ''' 
SELECT *
FROM sqlite_master
WHERE type = "table"
'''
pd.read_sql(query, cnx)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\nid_proveedor INT P...
1,table,piezas,piezas,4,CREATE TABLE piezas (\nid_pieza INT PRIMARY KE...


In [ ]:
query = '''
CREATE TABLE categoria_pieza (
id_categoria INT PRIMARY KEY,
nombre_categoria TEXT NOT NULL
) 
'''
cursor_prov.execute(query)

In [ ]:
query = ''' 
SELECT *
FROM sqlite_master
WHERE type = "table"
'''
pd.read_sql(query, cnx)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\nid_proveedor INT P...
1,table,piezas,piezas,4,CREATE TABLE piezas (\nid_pieza INT PRIMARY KE...
2,table,categoria_pieza,categoria_pieza,6,CREATE TABLE categoria_pieza (\nid_categoria I...


In [ ]:
query = ''' 
SELECT *
FROM categoria_pieza
'''
pd.read_sql(query, cnx)

,id_categoria,nombre_categoria


In [ ]:
query = '''
CREATE TABLE operaciones (
id_operacion INT PRIMARY KEY,
id_proveedor INT NOT NULL,
id_pieza INT NOT NULL,
fecha TEXT NOT NULL,
id_factura INT NOT NULL
) 
'''
cursor_prov.execute(query)

In [ ]:
query = ''' 
SELECT *
FROM sqlite_master
WHERE type = "table"
'''
pd.read_sql(query, cnx)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\nid_proveedor INT P...
1,table,piezas,piezas,4,CREATE TABLE piezas (\nid_pieza INT PRIMARY KE...
2,table,categoria_pieza,categoria_pieza,6,CREATE TABLE categoria_pieza (\nid_categoria I...
3,table,operaciones,operaciones,8,CREATE TABLE operaciones (\nid_operacion INT P...


In [ ]:
query = '''
CREATE TABLE facturas (
id_factura INT PRIMARY KEY,
id_proveedor INT NOT NULL,
id_pieza INT NOT NULL,
cantidad_piezas INT NOT NULL,
precio_ud INT NOT NULL,
total INT NOT NULL,
fecha_factura TEXT NOT NULL
) 
'''
cursor_prov.execute(query)

In [ ]:
query = ''' 
SELECT *
FROM sqlite_master
WHERE type = "table"
'''
pd.read_sql(query, cnx)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\nid_proveedor INT P...
1,table,piezas,piezas,4,CREATE TABLE piezas (\nid_pieza INT PRIMARY KE...
2,table,categoria_pieza,categoria_pieza,6,CREATE TABLE categoria_pieza (\nid_categoria I...
3,table,operaciones,operaciones,8,CREATE TABLE operaciones (\nid_operacion INT P...
4,table,facturas,facturas,10,CREATE TABLE facturas (\nid_factura INT PRIMAR...


In [ ]:
query = ''' 
SELECT *
FROM proveedores
'''
pd.read_sql(query, cnx)

,id_proveedor,nombre,direccion,ciudad,provincia


In [6]:
fake = Faker("es_ES")

In [222]:
import re
patron = r".*\n([a-zA-z]+.*),"

patron = r".*\n(.*),"

In [346]:
class Proveedor:
    id = None
    nombre = fake.name()
    direccion = fake.address()
    provincia = re.match(patron, direccion).group(1)
    ciudad = provincia
    


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="dario1")
dict_proveedores = {}
for i in range (1,21):
    dict_proveedores[f"Proveedor_{i}"] = Proveedor()
    dict_proveedores[f"Proveedor_{i}"].nombre = fake.name()
    dict_proveedores[f"Proveedor_{i}"].direccion = fake.street_address()
    dict_proveedores[f"Proveedor_{i}"].ciudad = fake.city()
    dict_proveedores[f"Proveedor_{i}"].provincia = geolocator.geocode(f"{dict_proveedores[f"Proveedor_{i}"].ciudad}").address.split(",")
    if len(dict_proveedores[f"Proveedor_{i}"].provincia)>= 3:
        dict_proveedores[f"Proveedor_{i}"].provincia = geolocator.geocode(f"{dict_proveedores[f"Proveedor_{i}"].ciudad}").address.split(",")[-3]
    else:
        dict_proveedores[f"Proveedor_{i}"].provincia = geolocator.geocode(f"{dict_proveedores[f"Proveedor_{i}"].ciudad}").address.split(",")[-2]
    if i < 10:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"10{i}")
    else:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"1{i}")

In [396]:
dict_proveedores = {}
for i in range (1,21):
    dict_proveedores[f"Proveedor_{i}"] = Proveedor()
    dict_proveedores[f"Proveedor_{i}"].nombre = fake.name()
    dict_proveedores[f"Proveedor_{i}"].direccion = fake.address()
    dict_proveedores[f"Proveedor_{i}"].ciudad = re.match(patron, dict_proveedores[f"Proveedor_{i}"].direccion).group(1)
    dict_proveedores[f"Proveedor_{i}"].provincia = dict_proveedores[f"Proveedor_{i}"].ciudad
    if i < 10:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"10{i}")
    else:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"1{i}")

In [397]:
for clave,valor in dict_proveedores.items():
    print(f"{clave}: {valor}")
    print(valor.ciudad, valor.provincia)

Proveedor_1: <__main__.Proveedor object at 0x0000019891DFAF20>
Jaén Jaén
Proveedor_2: <__main__.Proveedor object at 0x0000019891D4ABA0>
Cantabria Cantabria
Proveedor_3: <__main__.Proveedor object at 0x0000019891D499B0>
Ourense Ourense
Proveedor_4: <__main__.Proveedor object at 0x0000019891C8BCB0>
Murcia Murcia
Proveedor_5: <__main__.Proveedor object at 0x0000019891F45E10>
Jaén Jaén
Proveedor_6: <__main__.Proveedor object at 0x0000019891F46040>
Álava Álava
Proveedor_7: <__main__.Proveedor object at 0x0000019891F45D30>
Albacete Albacete
Proveedor_8: <__main__.Proveedor object at 0x0000019891F46350>
Segovia Segovia
Proveedor_9: <__main__.Proveedor object at 0x0000019891F45FD0>
Santa Cruz de Tenerife Santa Cruz de Tenerife
Proveedor_10: <__main__.Proveedor object at 0x0000019891F45BE0>
Madrid Madrid
Proveedor_11: <__main__.Proveedor object at 0x0000019891F457F0>
Segovia Segovia
Proveedor_12: <__main__.Proveedor object at 0x0000019891F454E0>
Vizcaya Vizcaya
Proveedor_13: <__main__.Proveedor

In [403]:
for clave, valor in dict_proveedores.items():
    query = f''' 
    INSERT INTO proveedores (id_proveedor, nombre, direccion, ciudad, provincia)
    VALUES ("{valor.id}", "{valor.nombre}", "{valor.direccion.replace("\n", ", ")}", "{valor.ciudad}", "{valor.provincia}")
    '''
    cursor_prov.execute(query)

In [404]:
query = ''' 
SELECT *
FROM proveedores
'''
pd.read_sql(query, cnx)

,id_proveedor,nombre,direccion,ciudad,provincia
0,101,Juanita Miralles Alcalde,"Pasaje Amalia Aguilar 18, Jaén, 07520",Jaén,Jaén
1,102,Belen Tapia Mateu,"Camino Isaac Delgado 327, Cantabria, 01723",Cantabria,Cantabria
2,103,Raquel Arrieta,"Callejón de Pili Santiago 99 Puerta 1 , Ourens...",Ourense,Ourense
3,104,Jacobo Toro Tena,"Calle Amílcar Morán 47 Puerta 9 , Murcia, 42128",Murcia,Murcia
4,105,Loreto Nadal,"Paseo de Brunilda Espada 29, Jaén, 15223",Jaén,Jaén
5,106,Mario Peiró Puente,"Via de Juan Luis Roca 75 Piso 6 , Álava, 14655",Álava,Álava
6,107,Flavia Garmendia Bru,"Ronda de Herminio Rodríguez 66, Albacete, 14568",Albacete,Albacete
7,108,Anselmo Custodio Matas Boada,"Camino Josefa Rovira 34, Segovia, 35128",Segovia,Segovia
8,109,Adelaida Catalá Torre,"Pasaje de Luciana Leal 98, Santa Cruz de Tener...",Santa Cruz de Tenerife,Santa Cruz de Tenerife
9,110,Encarnación Cerdán Esteban,"Acceso de Virgilio Tirado 82 Apt. 46 , Madrid,...",Madrid,Madrid


In [406]:
cnx.commit()